# Tennis PDI - Match Length Prediction Using A Regression Forest

In [ ]:
# This is the notebook used to generate our web app. For a detailed commented version, see regression-tree.ipynb !

In [2]:
### IMPORTS ###

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display

In [3]:
# !jupyter nbextension enable --py widgetsnbextension --sys-prefix
# !jupyter serverextension enable voila --sys-prefix

In [32]:
PLAYER1 = "Roger Federer"
PLAYER2 = "Rafael Nadal"

def_player1_text = widgets.Text(value="Roger Federer", placeholder="Player 1", description='Enter P1')
def_player2_text = widgets.Text(value="Rafael Nadal", placeholder="Player 2", description='Enter P2')

button = widgets.Button(description="Confirm")

out1 = widgets.Output()

def validate(b):
    global PLAYER1
    global PLAYER2
    out1.clear_output()
    if (def_player1_text.value in atp_players.name.values) and (def_player2_text.value in atp_players.name.values) :
        with out1 :
            print("Valid players.")
        PLAYER1 = def_player1_text.value
        PLAYER2 = def_player2_text.value
    else :
        with out1 :
            print("Invalid players. Try again.")

button.on_click(validate)

display(def_player1_text, def_player2_text, button, out1)

Text(value='Roger Federer', description='Enter P1', placeholder='Player 1')

Text(value='Rafael Nadal', description='Enter P2', placeholder='Player 2')

Button(description='Confirm', style=ButtonStyle())

Output()

In [5]:
### CLEAN PLAYER MATCHES TABLES ###

atp = pd.read_csv("atp_cat.csv")
# atp = atp[atp.loser_hand != 'U']
# atp.to_csv("atp_cat.csv", index=False)

player_won = atp[atp["winner_name"] == PLAYER1]
player_lost = atp[atp["loser_name"] == PLAYER1]

# We'll drop all of the player's information except his age and rank
player_won = player_won.drop(labels=["winner_name", "winner_hand", "winner_ht", "winner_ioc"], axis=1)
player_won = player_won.rename(columns={"winner_age": "player_age", "winner_rank": "player_rank", "winner_rank_points": "player_rank_points",
                                "loser_name":"opp_name", "loser_hand":"opp_hand", "loser_ht":"opp_ht", "loser_ioc":"opp_ioc", "loser_age":"opp_age",
                                "loser_rank": "opp_rank", "loser_rank_points": "opp_rank_points"})
player_won["index2"] = player_won.index
player_won["player_won"] = "1"

player_lost = player_lost.drop(labels=["loser_name", "loser_hand", "loser_ht", "loser_ioc"], axis=1)
player_lost = player_lost.rename(columns={"loser_age": "player_age", "loser_rank": "player_rank", "loser_rank_points": "player_rank_points",
                                    "winner_name":"opp_name", "winner_hand":"opp_hand", "winner_ht":"opp_ht", "winner_ioc":"opp_ioc", "winner_age":"opp_age",
                                    "winner_rank": "opp_rank", "winner_rank_points": "opp_rank_points"})
player_lost["index2"] = player_lost.index
player_lost["player_won"] = "0"

player = pd.concat([player_won, player_lost])
# player.drop(list(player.filter(regex = 'Unnamed')), axis = 1, inplace = True)
player.sort_index(inplace=True)

# Win streaks
result = player.player_won.astype(int)
consecutive = result.groupby((result != result.shift()).cumsum()).cumcount()
wins = pd.DataFrame({"win" : result, "consecutive" : consecutive})
m = wins.win == 1
wins.consecutive = wins.consecutive.where(m, 0)
player["consecutive"] = wins.consecutive

player.to_csv("player.csv", index=False)     # Saving the table for ease of use

atp_players = pd.read_csv("atp_players.csv")
atp_players['name'] = atp_players['name_first'] + ' ' + atp_players['name_last']
atp_players.to_csv("atp_players.csv", index=False)     # Saving the table for ease of use



### FORMATTING

player1_pf = pd.DataFrame(player[["minutes", "surface", "best_of", "opp_hand", "opp_ht", "opp_age", "tourney_level", "round"]])
player1_pf.loc[:,"rank_diff"] = player.loc[:,"player_rank"] - player.loc[:,"opp_rank"]
player1_pf.loc[:,"consecutive"] = player.loc[:,"consecutive"]

needed_columns = ['minutes', 'opp_ht', 'opp_age', 'rank_diff', 'consecutive',
       'surface_Clay', 'surface_Grass', 'surface_Hard', 
       'best_of_3', 'best_of_5', 'opp_hand_L', 'opp_hand_R', 
       'tourney_level_A', 'tourney_level_G', 'tourney_level_M', 
       'round_F', 'round_QF', 'round_R128', 'round_R16',
       'round_R32', 'round_R64', 'round_RR', 'round_SF']

# One-Hot Encoding
player1 = pd.get_dummies(data=player1_pf, columns=["surface", "best_of", "opp_hand", "tourney_level", "round"])
player1 = player1.dropna(axis=0, how='any')

if (set(needed_columns)).issubset(set(player1.columns.tolist())) :
       player1 = player1[needed_columns]
else :
       print("Player 1 doesn't have enough data")
       print (player1.columns)
       raise SystemExit(0)

In [6]:
### PRE-PROCESSING ###

from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

target='minutes'

scale_x = False
scale_y = False

def preprocessing(scale_x=False, scale_y=False) :

    # 1. X, y unscaled
    X = player1.drop([target], axis=1)
    y = np.asarray(player1[target])
    y = y.reshape(-1,1)

    xscaler = StandardScaler()
    yscaler = StandardScaler()

    if (scale_x) :
        print("Scaling X...")
        xscaler.fit(X[['opp_ht', 'opp_age', 'rank_diff']])
        X[['opp_ht', 'opp_age', 'rank_diff']] = xscaler.transform(X[['opp_ht', 'opp_age', 'rank_diff']])

    if (scale_y) :
        print("Scaling y...")
        yscaler.fit(y)
        y = yscaler.transform(y)

    return X, y, xscaler, yscaler

X, y, xscaler, yscaler = preprocessing(scale_x, scale_y)

,opp_ht,opp_age,rank_diff,consecutive,surface_Clay,surface_Grass,surface_Hard,best_of_3,best_of_5,opp_hand_L,...,tourney_level_G,tourney_level_M,round_F,round_QF,round_R128,round_R16,round_R32,round_R64,round_RR,round_SF
24174,180.0,23.731691,614.0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
24932,180.0,28.618754,833.0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,0
24941,196.0,28.418891,835.0,1,0,0,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
24946,183.0,28.457221,858.0,0,0,0,1,1,0,1,...,0,0,0,1,0,0,0,0,0,0
24954,180.0,28.435318,388.0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,0


,0
0,80.0
1,60.0
2,85.0
3,79.0
4,60.0


In [7]:
### PARAMETER SELECTION ###

from sklearn.model_selection import GridSearchCV

params = {
    # "criterion":("squared_error", "friedman_mse", "absolute_error", "poisson"), 
    "max_depth":np.arange(3, 6), 
    "min_samples_leaf":np.arange(1, 100), 
    # "min_weight_fraction_leaf":[0.5, 0.4, 0.3, 0.2, 0.1, 0.05, 0.075, 0.001], 
    # "max_features":np.arange(1, 25)
}

dtr = DecisionTreeRegressor(random_state=42)
dtr_cv = GridSearchCV(dtr, params, scoring="r2", n_jobs=-1, verbose=0, cv=10)

In [8]:
### CUSTOM PREDICTION ###

def custom_predict(dtr, scale_x, scale_y, height, age, rank_diff, consecutive, surface, best_of, hand, level, round) :
    # returns the predicted length of a match given a set of match conditions

    # surface : clay(1), grass(2), hard(3)
    # best of : 3(1), 5(2)
    # hand : left(1), right(2)
    # level : A(1), D(2), G(3), M(4)
    # round : F(1), QF(2), R128(3), R16(4), R32(5), R64(6), R(7), SF(8)

    surface_input = [0, 0, 0]
    surface_input[surface - 1] = 1

    bo_input = [0, 0]
    bo_input[best_of - 1] = 1

    hand_input = [0, 0]
    hand_input[hand - 1] = 1

    level_input = [0, 0, 0]
    level_input[level - 1] = 1

    round_input = [0, 0, 0, 0, 0, 0, 0, 0]
    round_input[round - 1] = 1

    X_custom = pd.DataFrame(columns=X.columns)


    if (not scale_x and not scale_y) :  # 1. X, y unscaled
        input = [height, age, rank_diff] + [consecutive] + surface_input + bo_input + hand_input + level_input + round_input
        X_custom = pd.DataFrame(columns=X.columns)
        print(input)
        X_custom.loc[0] = input
        display(X_custom)

        print("Prediction : ", dtr.predict(X_custom), " minutes")

        return(dtr.predict(X_custom)[0])

    elif (scale_x and scale_y) :    # 2. X, y scaled
        input = xscaler.transform([[height, age, rank_diff]])
        print(input)
        input = np.append(input[0], [consecutive] + surface_input + bo_input + hand_input + level_input + round_input)

        X_custom.loc[0] = input
        # print("Scaled input : ")
        # display(X_custom)

        print("Scaled Prediction : ", dtr.predict(X_custom), " minutes")
        print("Prediction : ", yscaler.inverse_transform(dtr.predict(X_custom)), " minutes")

        return(yscaler.inverse_transform(dtr.predict(X_custom))[0])

    elif (scale_x and not scale_y) :    # 3. X scaled, y unscaled
        input = xscaler.transform([[height, age, rank_diff]]).tolist()
        input = np.append(input[0], [consecutive] + surface_input + bo_input + hand_input + level_input + round_input)

        X_custom.loc[0] = input
        # print("Scaled input : ")
        # display(X_custom)

        print("Prediction : ", dtr.predict(X_custom)[0], " minutes")

        return(dtr.predict(X_custom)[0])


# e.g. custom_predict(dtr, scale_x, scale_y, 180, 22, 0, 0, 4, 1, 2, 4, 5)

In [9]:
### TREE VISUALIZATION ###

from IPython.display import Image
from six import StringIO
from sklearn.tree import export_graphviz
import pydot

def visualize_tree(dtr) :

    features = list(player1.columns)
    features.remove("minutes")

    dot_data = StringIO()
    export_graphviz(dtr, out_file=dot_data, feature_names=features, filled=True)
    graph = pydot.graph_from_dot_data(dot_data.getvalue())
    display(Image(graph[0].create_png()))

In [10]:
### INTERFACE ###

# Match Settings

match_settings_title = widgets.Label(value='MATCH CONDITIONS')

surface_radio = widgets.RadioButtons(
    options=['Hard', 'Grass', 'Clay'],
    description='Surface',
    disabled=False
)

best_of_radio = widgets.RadioButtons(
    options=['3', '5'],
    description='Best of',
    disabled=False
)

level_dropdown = widgets.Dropdown(
    description="Level",
    options=['Grand Slam (G)', 'Masters 1000s (M)', 'Other (A)']
)

round_dropdown = widgets.Dropdown(
    description="Round",
    options=['Final', 'Semifinals', 'Quarterfinals', 'R16', 'R32', 'R64', 'R128', 'R']
)

match_inputs_1 = widgets.HBox([surface_radio, best_of_radio])
match_inputs_2 = widgets.HBox([level_dropdown, round_dropdown])

# Players

player1_title = widgets.Label(value='PLAYER 1')

player1_text = widgets.Text(value=PLAYER1, placeholder="Player 1", description='Name')

player1_height_display = widgets.Text(description='Height')
player1_hand_display = widgets.Text(description='Hand')
player1_rank_display = widgets.Text(description='Rank')
player1_age_display = widgets.Text(description='Age')
player1_cons_display = widgets.Text(description='Win streak')

if (pd.notna(atp_players.loc[atp_players.name==PLAYER1, 'height'].values[0])) :
    player1_height_display.value = str(atp_players.loc[atp_players.name==PLAYER1, 'height'].values[0])
else :
    player1_height_display.value = '185'
player1_hand_display.value = str(atp_players.loc[atp_players.name==PLAYER1, 'hand'].values[0])
player1_rank_display.value = str(atp_players.loc[atp_players.name==PLAYER1, 'rank'].values[0])
player1_age_display.value = '25'
player1_cons_display.value = '0'

player1_widgets = widgets.VBox([player1_title, player1_text, player1_height_display, player1_hand_display, player1_rank_display, player1_age_display, player1_cons_display])


player2_title = widgets.Label(value='PLAYER 2')

player2_text = widgets.Text(value=PLAYER2, placeholder="Player 2", description='Name')

player2_height_display = widgets.Text(description='Height')
player2_hand_display = widgets.Text(description='Hand')
player2_rank_display = widgets.Text(description='Rank')
player2_age_display = widgets.Text(description='Age')
player2_cons_display = widgets.Text(description='Win streak')

if (pd.notna(atp_players.loc[atp_players.name==PLAYER2, 'height'].values[0])) :
    player2_height_display.value = str(atp_players.loc[atp_players.name==PLAYER2, 'height'].values[0])
else :
    player2_height_display.value = '185'
player2_hand_display.value = str(atp_players.loc[atp_players.name==PLAYER2, 'hand'].values[0])
player2_rank_display.value = str(atp_players.loc[atp_players.name==PLAYER2, 'rank'].values[0])
player2_age_display.value = '25'
player2_cons_display.value = '0'

player2_widgets = widgets.VBox([player2_title, player2_text, player2_height_display, player2_hand_display, player2_rank_display, player2_age_display, player2_cons_display])


player_inputs = widgets.HBox([player1_widgets, player2_widgets])

display(match_settings_title, match_inputs_1, match_inputs_2)
display(player_inputs)
print("(Optional) For better precision, input player age and win streak.")



### BEHAVIOR ###

def player1_eventhandler(change):
    global PLAYER1
    if change.new in atp_players.name.values :
        PLAYER1 = change.new
        if (pd.notna(atp_players.loc[atp_players.name==PLAYER1, 'height'].values[0])) :
            player1_height_display.value = str(atp_players.loc[atp_players.name==PLAYER1, 'height'].values[0])
        else :
            player1_height_display.value = '185'
        player1_hand_display.value = str(atp_players.loc[atp_players.name==PLAYER1, 'hand'].values[0])
        player1_rank_display.value = str(atp_players.loc[atp_players.name==PLAYER1, 'rank'].values[0])
        
def player2_eventhandler(change):
    global PLAYER2
    if change.new in atp_players.name.values :
        PLAYER2 = change.new
        if (pd.notna(atp_players.loc[atp_players.name==PLAYER2, 'height'].values[0])) :
            player2_height_display.value = str(atp_players.loc[atp_players.name==PLAYER2, 'height'].values[0])
        else :
            player2_height_display.value = '185'
        player2_hand_display.value = str(atp_players.loc[atp_players.name==PLAYER2, 'hand'].values[0])
        player2_rank_display.value = str(atp_players.loc[atp_players.name==PLAYER2, 'rank'].values[0])

player1_text.observe(player1_eventhandler, names='value')
player2_text.observe(player2_eventhandler, names='value')

Label(value='MATCH CONDITIONS')

(Optional) For better precision, input player age and win streak.


In [11]:
# Converting the user inputs into inputs usable by the custom_predict function

sd = { 'Clay':1, 'Grass':2, 'Hard':3 }                                                                  # Surface dictionary
bod = { '3':1, '5':2 }                                                                                  # Best of dictionary
hd = { 'L':1, 'R':2 }                                                                                   # Hand dictionary
ld = { 'Other (A)':1, 'Grand Slam (G)':2, 'Masters 1000s (M)':3 }    # Level dictionary
rd = { 'Final':1, 'Quarterfinals':2, 'R128':3, 'R16':4, 'R32':5, 'R64':6, 'R':7, 'Semifinals':8 }       # Round dictionary

def t2i(dict, text) :   # text to input
    return dict[text]

ht = float(player2_height_display.value)
a = float(player2_age_display.value)
rk = float(player1_rank_display.value) - float(player2_rank_display.value)
c = int(player1_cons_display.value)
s = t2i(sd, surface_radio.value)
bo = int(t2i(bod, best_of_radio.value))
h = t2i(hd, player2_hand_display.value)
l = t2i(ld, level_dropdown.value)
r = t2i(rd, round_dropdown.value)

In [37]:
### REGRESSION FOREST ###

test_scores = []
predictions = []
average_prediction = -1

iter = 10

# UI

iter_label = widgets.Label('Nb of trees')
iter_slider = widgets.IntSlider(value=iter, min=2, max=100)
iter_text = widgets.Text()
widgets.jslink((iter_slider, 'value'), (iter_text, 'value'))
iter_input = widgets.HBox([iter_label, iter_slider, iter_text])

loading_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=iter,
    bar_style='',
    orientation='horizontal'
)
loading_label = widgets.Label(value='Planting Trees...')
loading = widgets.HBox([loading_label, loading_bar])

# The Loop

def random_forest_loop() :
    
    global average_prediction

    for i in range(iter) :

        Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=.3)
        dtr_cv.fit(Xtrain, ytrain)

        best_params = dtr_cv.best_params_
        print(f"Best parameters: {best_params})")

        dtr = DecisionTreeRegressor(**best_params)
        dtr.fit(Xtrain, ytrain)

        # Testing :

        ypred = dtr.predict(Xtest)

        print(f"Training score {i+1} : ", dtr.score(Xtrain, ytrain))
        print(f"Testing score {i+1} : ", dtr.score(Xtest, ytest))
        test_scores.append(dtr.score(Xtest, ytest))

        predictions.append(custom_predict(dtr, scale_x, scale_y, ht, a, rk, c, s, bo, h, l, r))     # ???

        # visualize_tree(dtr)

        loading_bar.value += 1
        if (i==iter-1) :
            loading_bar.bar_style='success'


        print("------------------------------------------------------------------------------------------------")

    print("------------------------------------------------------------------------------------------------")
    print("Test scores : ", test_scores)
    print("Test scores average : ", sum(test_scores)/len(test_scores))

    average_prediction = sum(predictions) / len(predictions)

    print("------------------------------------------------------------------------------------------------")
    print(f"Predictions : {predictions}")
    print(f"Average predicted length : {average_prediction}")


launch_btn = widgets.Button(description='Plant Forest')

out2 = widgets.Output()

def iter_slider_eventhandler(change) :
    global iter
    iter = change.new

def launch_btn_onclick(change) :
    out2.clear_output()
    loading_bar.max = iter
    with out2 : 
        display(loading)
    random_forest_loop()

iter_slider.observe(iter_slider_eventhandler, names='value')
launch_btn.on_click(launch_btn_onclick)

display(iter_input)
display(launch_btn)
display(out2)



### HISTOGRAM ###

# ATP Colors
# Dark blue : #002865
# Light blue : #00AFF0
# Rolland Garros Orange : #CB5A19
# Tennis Ball Yellow : #8CFF00

time_step = 15  # precision : 15/20/30 minutes

def plot_distribution(time_step) : 
    
    fig, ax = plt.subplots(1, figsize=(18,6))

    plt.suptitle(f'Distribution of Match Length Predictions : {PLAYER1} v. {PLAYER2}', fontweight='bold')
    plt.xlabel("Minutes")
    # plt.ylabel("Nb of predictions")

    ax.spines['bottom'].set_visible(True)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.yticks([])
    
    
    intervals=np.arange(0, 300, time_step)
    n, bins, patches = plt.hist(predictions, bins=intervals, color='#00AFF0')

    plt.xticks(bins)
    plt.grid(color='white', lw = 1, axis='x')

    xticks = [(bins[idx+1] + value)/2 for idx, value in enumerate(bins[:-1])]

    for idx, value in enumerate(n) :
        if value > 0 :
            plt.text(xticks[idx], value * 1.05, f"{int(100 * value / iter)}%", ha='center')
            plt.text(xticks[idx], value / 2, int(value), ha='center', color='w', fontweight='semibold')
        
    if (average_prediction >= 0) : plt.axvline(x=average_prediction, color='#CB5A19')
    # plt.axvline(x=116, color='#8CFF00')
    # plt.legend()


    plt.show()

out3 = widgets.Output()
time_step_slider = widgets.IntSlider(value=time_step, min=5, max=45, step=5, description='Time Step')

with out3:
    plot_distribution(time_step)

def time_step_slider_eventhandler(change) :
    out3.clear_output()
    with out3:
        plot_distribution(change.new)

time_step_slider.observe(time_step_slider_eventhandler, names='value')

print("Drag slider to plot :")
display(time_step_slider, out3)

Button(description='Plant Forest', style=ButtonStyle())

Output()

IntSlider(value=15, description='Time Step', max=45, min=5, step=5)

Output()

IntSlider(value=15, description='Time Step', max=45, min=5, step=5)

Output()

In [ ]:
reset_btn = widgets.Button(description="Reset")

def reset_click(change):
    global test_scores
    global predictions
    global average_prediction
    test_scores = []
    predictions = []
    average_prediction = -1
    
    out1.clear_output()
    out2.clear_output()
    out3.clear_output()
    
    
reset_btn.on_click(reset_click)

display(reset_btn)